In [1]:
import os
import pandas as pd
import cv2


In [2]:
# Define paths to dataset folders
train_folder = "./Fish.v1-416x416.tensorflow/train"
valid_folder = "./Fish.v1-416x416.tensorflow/valid"
test_folder = "./Fish.v1-416x416.tensorflow/test"

In [7]:
# Function to read annotation files
def read_annotations(folder):
    annotation_file = os.path.join(folder, "_annotations.csv")
    return pd.read_csv(annotation_file)

def crop_images(folder, annotations):
    for index, row in annotations.iterrows():
        filename_col = annotations.columns[0]  # Assuming the first column is the filename
        filename = row[filename_col]
        image_path = os.path.join(folder, filename)
        try:
            image = cv2.imread(image_path)
            if image is None:
                print(f"Error: Unable to read image '{filename}'. Skipping...")
                continue
        except Exception as e:
            print(f"Error: Unable to read image '{filename}': {str(e)}. Skipping...")
            continue
        class_name = row['class']  # Adjusted column name to lowercase
        class_folder = os.path.join(folder, class_name)
        if not os.path.exists(class_folder):
            os.makedirs(class_folder)
        xmin, ymin, xmax, ymax = row['xmin'], row['ymin'], row['xmax'], row['ymax']  # Adjusted column names to lowercase
        if xmin == xmax or ymin == ymax:  # Check if width or height is zero
            print(f"Error: Bounding box has zero width or height in image '{filename}'. Skipping...")
            continue
        cropped_img = image[ymin:ymax, xmin:xmax]
        if cropped_img.shape[0] == 0 or cropped_img.shape[1] == 0:
            print(f"Error: Cropped image '{filename}' is empty. Skipping...")
            continue
        cv2.imwrite(os.path.join(class_folder, filename), cropped_img)


In [8]:
# Process train dataset
train_annotations = read_annotations(train_folder)
crop_images(train_folder, train_annotations)

# Process validation dataset
valid_annotations = read_annotations(valid_folder)
crop_images(valid_folder, valid_annotations)

# Process test dataset
test_annotations = read_annotations(test_folder)
crop_images(test_folder, test_annotations)

Error: Bounding box has zero width or height in image 'FishDataset554_png.rf.b2XgzOC8YWtCgAaV6mbC.jpg'. Skipping...
Error: Bounding box has zero width or height in image 'FishDataset554_png.rf.27536e4740fb6e7e0324302187967adb.jpg'. Skipping...
